NOTE: This code is of significant length and will take several hours to fully run on a regular computer. We suggest running the "Import & Install Necessary Packages" and "Data Preprocessing & New DataSet Creation" sections prior to deciding what types of analyses should be run.

We perform different types of analysis with several subsection for each:
1. NetworkX Graph Analysis
2. Prediction of wages and instigator status
3. PCA & MCA Clustering
4. Additional Code

# Import & Install Necessary Packages

In [ ]:
!pip install graspologic # for graph analysis https://graspologic.readthedocs.io/en/latest/
!pip install mca # PCA for categorical variables https://pypi.org/project/mca/ 
# !pip install graph_tool # for graph analysis https://graph-tool.skewed.de/ 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import math
import pymc3 as pymc
import networkx as nx
import graspologic
import mca
import sklearn
import sklearn.metrics as m
import sklearn.model_selection as ms
# import graph_tool
%matplotlib inline

# Import DataProcessing & Analysis Packages
from scipy.sparse import csr_matrix
from pandas import DataFrame
from collections import defaultdict
from graspologic.plot import heatmap


# Import Sklearn packages
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, f1_score
from sklearn.linear_model import LinearRegression, LassoCV, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.naive_bayes import BernoulliNB,  MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

# Import Keras packages for Neural Network 
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, Input, Dropout, GRU, Activation
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM
from keras import backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, CSVLogger, History

# Data Preprocessing & New DataSet Creation

## Clean Officer Data Set

In [ ]:
# Quantify Officer Ranks based on NYPD rank strucutre
# Note: second set is a repeated with space at end to account for weird data
#       set structuring
rank_dict = {
  'Police Officer': 1, 
  'Detective Third Grade': 2,
  'Detective Second Grade': 3, 
  'Detective First Grade': 4,
  'Sergeant': 4, 
  'Detective Specialist': 4, 
  'Lieutenant': 5,
  'Captain': 6,
  'Deputy Inspector': 7,  
  'Inspector': 8, 
  'Deputy Chief Inspector': 9, 
  'Assistant Chief Inspector': 10,
  'Chief': 11, 
  'Deputy Commissioner': 12, 
  'Assistant Deputy Commissioner': 13,
  'Assistant Commissioner': 14,
  'Commissioner': 15,
    
  'Police Officer ': 1, 
  'Detective Third Grade ': 2,
  'Detective Second Grade ': 3, 
  'Detective First Grade ': 4,
  'Sergeant ': 4, 
  'Detective Specialist ': 4, 
  'Lieutenant ': 5,
  'Captain ': 6,
  'Deputy Inspector ': 7,  
  'Inspector ': 8, 
  'Deputy Chief Inspector ': 9, 
  'Assistant Chief Inspector ': 10,
  'Chief ': 11, 
  'Deputy Commissioner ': 12, 
  'Assistant Deputy Commissioner ': 13,
  'Assistant Commissioner ': 14,
  'Commissioner ': 15,
}


In [ ]:
file2 = r"capstat_expanded_dataset.csv"
officers = pd.read_csv(file2)

In [ ]:
# Clean Names for Officers
# Truncate names (to match naming convention of CCRB data set)

shortF = [] # Shortened first name
shortL = [] # Shortened last name
shortAll = [] # Shortened full name

for name in officers['First Name']:
  # Truncate first names of over 10 characters
    if len(name) > 10: shortF.append(name[0 : 10])
    else: shortF.append(name)
        
for name in officers['Last Name']:
    # Truncate last names of over 15 characters
    if len(name) > 15: shortL.append(name[0 : 10])
    else: shortL.append(name)
        
for i in range(0,len(shortF)):
    shortAll.append(shortF[i] + ' ' + shortL[i])
    
# Substitute names in Officer data set
officers["Full"] = shortAll

## Clean Complaint Data Set

In [ ]:
file1 = r"CCRB_database_raw.csv"
df = pd.read_csv(file1) # Complaint data set
df = df.drop(['ShieldNo', 'AsOfDate'], axis=1)

# Fix up some common typos in the dataset. (Given in starter notebook)
allegations_correction_dict = {
    'Vehicle Searched': 'Vehicle search',
    'Other - Ethnic Slur': 'Ethnic Slur',
    'Refusal to provide shield number': 'Refusal to provide name/shield number',
    'Refusal to provide name': 'Refusal to provide name/shield number',
    'Property Damaged': 'Property damaged',
    'Gun Pointed': 'Gun pointed/gun drawn',
    'Gun pointed': 'Gun pointed/gun drawn',
    'Threat of Arrest': 'Threat of arrest',
    'Other- Discourtesy': 'Discourtesy',
    'Premise Searched': 'Premises entered and/or searched',
    'Entry of Premises': 'Premises entered and/or searched',
    'Search of Premises': 'Premises entered and/or searched',
    'Threat of Summons': 'Threat of summons',
    'Property Damaged': 'Property damaged',
    'Flashlight As Club': 'Flashlight As club',
    'Radio As Club': 'Radio as club',
    'Gun Fired': 'Gun fired',
    'Gun As Club': 'Gun As club',
}

penalty_correction_dict = {
    'Terminated': 'Termination',
}

# Additional Replacements (included in starter notebook)
df['Allegation'] = df['Allegation'].replace(allegations_correction_dict)
df['PenaltyDesc'] = df['PenaltyDesc'].replace(penalty_correction_dict)
df['Full Name'] = df['First Name'] + " " + df['Last Name']
df['Incident Date'] = pd.to_datetime(df['Incident Date'], format='%m/%d/%Y')
dates = df["Incident Date"]

df['day'] = df['Incident Date'].dt.day
df['month'] = df['Incident Date'].dt.month
df['year'] = df['Incident Date'].dt.year

df = df.drop(['Incident Date'], axis=1)


## Create & Export New Data Set

### Column names (sparse matrix)

In [ ]:
# All Unique Allegation Types - to create spare matrix
allegations = ['2Force', 'Beat', 'Vehicle search', 'Nasty Words', 'Detention',
       'Ethnic Slur', 'Black', 'Punch/Kick', 'Dragged/Pulled', 'Animal', 'Refusal to provide name/shield number', 'Word', 'Question',
       'Physical force', 'Hit against inanimate object',
       'Property damaged', 'Threat of force (verbal or physical)', 'Stop',
       'Frisk', 'Gun pointed/gun drawn', 'Threat of arrest',
       'Threat to notify ACS', 'Nightstick as club (incl asp & baton)',
       'Question and/or stop', 'Other - Force', 'Curse', 'Push/Shove',
       'Other - Abuse', 'Threat of force', '2Discourtesy',
       'Premises entered and/or searched', 'Forcible Removal to Hospital',
       'Refusal to process civilian complaint', 'Slap', 'Race',
       'Threat of summons', 'Vehicle stop', 'Search (of person)', 'Mace',
       'Retaliatory arrest', 'Vehicle', 'Retaliatory summons',
       'Threat to damage/seize property', 'Ethnicity',
       '2Abuse of Authority', 'Demeanor/tone', 'Other',
       'Nonlethal restraining device', 'Person Searched',
       'Nightstick/Billy/Club', 'Other blunt instrument as a club',
       'Strip-searched', 'Pepper spray', 'Interference with recording',
       'Failure to provide RTKA card', 'Sexual orientation',
       'Frisk and/or search', 'Chokehold', 'Gun Drawn',
       'Handcuffs too tight', 'Refusal to obtain medical treatment',
       'Flashlight As club', 'Threat to Property', 'Seizure of property',
       'Arrest/D. A. T.', 'Physical disability', 'Sexist Remark',
       'Radio as club', 'Police shield', 'Hispanic', 'Oriental',
       'Gun fired', 'Religion', 'Action', 'Gender', 'Sh Refuse Cmp',
       'Rude Gesture', 'Jewish', 'Gesture',
       'Refusal to show search warrant', 'Gun As club',
       'Flashlight as club', 'Gun as club', 'Profane Gesture',
       'Restricted Breathing', 'Threat re: removal to hospital',
       'Electronic device information deletion',
       'Search of recording device',
       'Sex Miscon (Sexual/Romantic Proposition)', 'Property Seized',
       'Photography/Videography', 'White', 'Arrest/Onlooker',
       'Other Asian', 'Gay/Lesbian Slur',
       'Improper dissemination of medical info',
       'Sex Miscon (Sexual Harassment, Gesture)',
       'Sex Miscon (Sexual Harassment, Verbal)',
       'Sexual Misconduct (Sexual Humiliation)',
       'Refusal to show arrest warrant', 'Threat re: immigration status',
       'Body Cavity Searches', 'Failed to Obtain Language Interpretation',
       'Gender Identity', 'Questioned immigration status',
       'Sex Miscon (Sexually Motivated Frisk)',
       'Sex Miscon (Sexually Motiv Strip-Search)']

# The Four FADO Allegation Types
fadoAllegations = ['Force', 'Abuse of Authority', 'Discourtesy', 'Offensive Language']

# All unique litigation results
results = ['Unsubstantiated', 'Complainant Unavailable', 'Unfounded',
       'Exonerated', 'Alleged Victim Unavailable',
       'Complainant Uncooperative',
       'Substantiated (Command Lvl Instructions)',
       'Substantiated (Formalized Training)', 'Complaint Withdrawn',
       'Miscellaneous - Subject Retired', 'Alleged Victim Uncooperative',
       'Substantiated (Charges)', 'Substantiated (Command Discipline)',
       'Victim Unidentified', 'Substantiated (Command Discipline B)',
       'Substantiated (Command Discipline A)', 'Miscellaneous',
       'Substantiated (No Recommendations)',
       'Closed - Pending Litigation', 'Substantiated (Instructions)',
       'Miscellaneous - Subject Resigned',
       'Miscellaneous - Subject Terminated', 'Officer(s) Unidentified',
       'Witness Uncooperative', 'Substantiated (MOS Unidentified)',
       'Witness Unavailable']

# The years our data set should cover
# NOTE: Can be adapted to any set of years for further analysis
# Reasoning for choosing 2000 - 2019 mentioned in the paper
years = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010','2011','2012', '2013', '2014', '2015', '2016', '2017', '2018']


In [ ]:
# Create copy of complaints & keep complaints betweeen 2000 and 2018 only (reasoning in paper)
copyDf = df.copy()
copyDf = copyDf[copyDf.year >= 2000]
copyDf = copyDf[copyDf.year <= 2018]
copyDf =   copyDf.loc[:, (copyDf != copyDf.iloc[0]).any()]  # remove 0 variance

### Extract officer information

In [ ]:
# Create NEW & improved DataFrame
newDf = pd.DataFrame(columns = ['Name', 'Rank', 'County', 'StartYear','rk1', 'rk1yr', 'rk2', 'rk2yr', 'rk3', 'rk3yr', 'wg18', 'wg17', 'wg16', 'wg15', 'wg14', 'wg13', 'wg12', 'wg11', 'wg10', 'wg09', 'wg08', 'wg07', 'wg12', 
                                   'wg06', 'wg05', 'wg04', 'wg03', 'wg02', 'wg01', 'wg00'])


# CLEAN OFFICERS DATABASE
# seperate wages, promotions into sparse matrix 

arr3 = []

# iterrate through officer data set and extract name
for row in officers.iterrows():
    if type(row[1][10]) == float:
        continue # if nan -> skip
   
   # extract name, district, current rank, and start year
    arr2 = []
    arr2.append(row[1][13])  
    arr2.append(row[1][4])
    arr2.append(row[1][7])
    arr2.append(re.search(r", (\d{4})", row[1][8]).group(1)) # 4 digit start year

    # Extract rank history (format e.g.: [FY2018][Detective] [FY2002][Officer])
    strRank = row[1][9] 

    # iterate through splits to extract data into array form
    strRk = strRank.split('[')
    index = 1

    # assuming up to 5 promotions (no officer has more than 4)
    for i in range(1, 5):
        arr2.append(strRk[index][2:-1])
        index += 1
        arr2.append(strRk[index].replace(']', ''))
        index += 1
        if index >= len(strRk):
            while i < 5:
                arr2.append(0)
                arr2.append(0)
                i += 1
            break
   
    while len(arr2) < 14:
        arr2.append(0)
    
    # Extract wage history ( format e.g. [FY2018][$1,000] [FY2019][$1,000])
    strWg = row[1][10]

    # iterate through splits to extract data into array form
    strWg = strWg.split('[')
    index = 1
    year = int(strWg[index][2:-1]) # last wage year

    # remove wages for years past 2018 (reasoning in paper)
    while year > 2018:
        index += 2
        if index > len(strRk):
            break
        year = int(strWg[index][2:-1])
    
    # Extract wages for 2000 to 2018 range (if NA/unemployed -> 0)
    counter = 0
    for i in range(2018, 1999, -1):
    
        if i == year:
            index += 1
            arr2.append(strWg[index][1:-1].replace(',', '').replace(']', '').replace(' ', ''))
            counter += 1
            index += 1
            if index >= len(strWg):
                continue
            year = int(strWg[index][2:-1])
        else:
            arr2.append(0)
            counter += 1
  
    # append to final data set
    arr3.append(arr2)

### Clean officer information

In [ ]:
# Create new dataframe using extracted officer data & name columns
newDf = pd.DataFrame(arr3, columns = ['Name', 'Rank', 'County', 'StartYear','rk1', 'rk1yr', 'rk2', 'rk2yr', 'rk3', 'rk3yr', 'rk4', 'rk4yr', 
                                 'rk5', 'rk5yr',  'wg18', 'wg17', 'wg16', 'wg15', 'wg14', 'wg13', 'wg12', 'wg11', 'wg10', 'wg09', 'wg08', 'wg07', 
                                   'wg06', 'wg05', 'wg04', 'wg03', 'wg02', 'wg01', 'wg00'])

# Remove name duplicates due to complexity in matching (reasoning in paper)
newDf.drop_duplicates(subset=['Name'], keep=False)

# Use rank dictionary to number rank structure
newDf = newDf.replace({"Rank": rank_dict})
newDf = newDf.replace({"rk1yr": rank_dict})
newDf = newDf.replace({"rk2yr": rank_dict})
newDf = newDf.replace({"rk3yr": rank_dict})
newDf = newDf.replace({"rk4yr": rank_dict})

# Keep track of total
newDf['CompNum'] = 0

# Rank throughout the years
for year in years:
    newDf['rk' + year] = 0

# Promotions throughout the years
for year in years:
    newDf['pr' + year] = 0

# Extract data into sparse matrix and fill out rank & promotions for every year
# Promotion is defined as: change in year to year rank seniority 
# (can be negative if demoted)

for row in newDf.iterrows():
    index = row[0]
    rankYr =int(row[1][4])
    rank = row[1][5]
    year = 2018
    while year >= rankYr:
        if year < 2000:  break
        newDf['rk' + str(year)][index] = rank
        year -= 1
    
    prevRank = rankYr
    rankYr = int(row[1][6])
    rank = row[1][7]
    if rankYr == 0:
        continue
    newDf['pr' + str(prevRank)][index] = row[1][5] - rank # change in ranks
    
    while year >= rankYr:
        if year < 2000:  break
        newDf['rk' + str(year)][index] = rank
        year -= 1
    
    prevRank = rankYr
    rankYr = int(row[1][8])
    rank = row[1][9]
    if rankYr == 0:
        continue
    newDf['pr' + str(prevRank)][index] = row[1][7] - rank
    
    while year >= rankYr:
        if year < 2000:  break
        newDf['rk' + str(year)][index] = rank
        year -= 1
      
    prevRank = rankYr
    rankYr = int(row[1][10])
    rank = row[1][11]
    if rankYr == 0:
        continue
    newDf['pr' + str(prevRank)][index] = row[1][9]- rank
    while year >= rankYr:
        if year < 2000:  break
        newDf['rk' + str(year)][index] = rank
        year -= 1

### Match Complaint Data to Officers

In [ ]:
# Match Complaint Data to individual officers

# Keep track of # of complaints matched
found = 0
notFound = 0

# Number of complaints against officer in given year
for year in years:
    newDf[year] = 0

# Number of FADO complaint types against officer in total
for alleg in fadoAllegations:
    newDf[alleg] = 0
  
# Number of results (e.g. exonerated) against officer in total
for result in results:
    newDf[result] = 0

# Number of specific complaint types against officer in total
for alleg in allegations:
    newDf[alleg] = 0

# For every complaint if name matches officer dataset extract data 
for row in copyDf.iterrows():
    name = row[1][11]
    indices = newDf[newDf['Name']== name].index.values # Name match

    # Does name match officer data set?
    if len(indices) > 0:
        index = indices[0]
        fado = row[1][6]
        allegation = row[1][7]
        result = row[1][8]
        date = str(int(row[1][14]))
        
        # Insert information unless nan
        if type(fado) != float:
            newDf[fado][index] += 1
        
        if type(allegation) != float:
            if((allegation != 'Force') and (allegation != 'Abuse of Authority')) and (allegation != 'Discourtesy'):
                    newDf[allegation][index] += 1
            else:
                newDf['2'+allegation][index] += 1
        if type(result) != float:
            newDf[result][index] += 1
            
        newDf[date][index] += 1 # Total complaints in year + 1
        newDf['CompNum'][index] += 1 # Total complaints + 1
        found += 1
        
    else:
        notFound += 1

# Print number found
print(found)
print(notFound)


###  Export Data Set

In [ ]:
# Export New Data Set 
newDf.to_csv('cleanData.csv')

# Make a copy of newDf and drop non-numeric values
xNew = newDf.copy()
xNew = xNew.drop(['Name'], axis=1)
xNew = xNew.drop(['County'], axis=1)
xNew = xNew.apply(pd.to_numeric) #transform to numeric

# OPTIONAL: Export CSV file of only numeric entries
# xNew.to_csv('cleanDataNew.csv') 

# NetworkX Graph

### Create Graph

In [ ]:
# Create Network Graph
G = nx.Graph()

# Sot by complaint Id  to find partners in same complaint
df = df.sort_values(by="Complaint Id")

# Iterate through complaints and identify partners
# Add edges between partners
prev = 0
partners = []
for index, row in df.iterrows():
  currPerson  = row["Unique Id"] # Unique Id
  curr = row['Complaint Id'] # Complaint Id
  G.add_node(currPerson)
  if  curr == prev:
    if currPerson not in partners:
      for person in partners:
        G.add_edge(person, currPerson) # update weights  
  else:
     partners = []
  
  if currPerson not in partners:
    partners.append(currPerson)
  prev = row['Complaint Id']

In [ ]:
# KModes
km = KModes(n_clusters=4, init='Huang', n_init=5, verbose=1)
dfCopy2['Command'] =dfCopy2['Command'].fillna('1')
dfCopy2 = dfCopy2.fillna('1')
clusters = km.fit_predict(dfCopy2)

# Print the cluster centroids
print(km.cluster_centroids_)
print(kp.labels_)

In [ ]:
# Heatmap of Graph

plt.subplot(121)
print(heatmap(G, cbar= False, title ='MMSBM Simulation'))
nbunch = [16205, 56121, 39959]
nx.draw(G.subgraph(nbunch))
# nx.draw(G)
'''
preds = nx.jaccard_coefficient(G, [(16205, 56121), (39959, 72558)])
for u, v, p in preds:
  print('(%d, %d) -> %.8f' % (u, v, p))
'''

# print(preds)
print(sorted(G.degree, key=lambda x: x[1], reverse=True))
# print(nx.clustering(G))
# plt.subplot(122)

nx.draw_random(G)
plt.subplot(222)

### Add degree info to new data set

Sort Graph by ID and match officers with their outgoing edge degrees & add findings to our new data set.

In [ ]:
# sort by degree
Gsorted = sorted(G.degree, key=lambda x: x[1], reverse=True)
print(Gsorted)

In [ ]:
graphdf = DataFrame(Gsorted,columns=['Unique Id','Degree'])
print(graphdf)

In [ ]:
graphdf['Name']=0
for index in range(0,len(graphdf)):
    indices = df.index[df['Unique Id'] == graphdf['Unique Id'][index]].tolist()
    graphdf['Name'][index]=df['Full Name'][indices[0]] 

In [ ]:
# Add degree to newDf
newDf['degree'] = 0
for index in range(0,len(graphdf)):
    name = graphdf['Name'][index]
    degree = graphdf['Degree'][index]
    indices = newDf[newDf['Name']== name].index.values
    if len(indices) > 0:
        newDf['degree'][indices[0]] = degree
    

In [ ]:
# Create binary variable "bad apple" for officers 
# with more than 4 outgoing edges

newDf['badApple'] = 0
for row in newDf.iterrows():
    index = row[0]
    degree = row[1][228]
    if degree <= 4: newDf['badApple'][index] = 0
    else: newDf['badApple'][index] = 1

# Optional: Value Counts
# newDf['degree'].value_counts()

### Predict Bad Apples

#### Linear Regression (cont.)

In [ ]:
# Remove any leakage and non-numeric values
# Perform Feature Selection
xNew = newDf.drop(['degree'], axis=1)
xNew = xNew.drop(['badApple'], axis=1)
xNew = xNew.drop(['County'], axis=1)
xNew = xNew.drop(['Name'], axis=1)

years = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18']

# Drop allegations (feature selection)
for alg in allegations:
    xNew = xNew.drop([alg], axis=1)
    # Optional Remove wages:
    # xNew = xNew.drop(['wg' + year], axis=1)

# Split in train, test sets
xPred = newDf['degree']
X_train, X_test, y_train, y_test = train_test_split(
    xNew, xPred, test_size=0.2, random_state=42)

linreg = LogisticRegression()
lin_params = [{'fit_intercept': [True, False]}]

# Perform Grid Search
grid = GridSearchCV(linreg,
                         param_grid=lin_params,
                         scoring='neg_mean_squared_error', #sklearn optimizing by maximizing negative MSE
                         n_jobs=1,
                         verbose=2,
                         cv=5)
grid.fit(X_train, y_train)
prediction_df = grid.predict(X_test)

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
prediction_df = regr.predict(X_test)
# The coefficients
# print('Coefficients: \n', grid.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, prediction_df))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, prediction_df))

columns = xNew.columns

# Print Coefficients with column name
for i in range(0, len(columns)):
    print(columns[i] + " " + str(regr.coef_[i]))


#### Additional Feature Selection

In [ ]:
# Perform additional Feature selection and test on NB

clf = ExtraTreesClassifier(n_estimators=10)
clf = clf.fit(xNew, xPred)
print(clf.feature_importances_)

model = SelectFromModel(clf, prefit=True)
X_new = model.transform(xNew) 

X_train, X_test, y_train, y_test = train_test_split(
    X_new, xPred, test_size=0.2, random_state=42)

# Choose Features here

# Perform BNB and test 
bnb = BernoulliNB()
bnb.fit(X_train, y_train)
bnbScore = bnb.score(X_test, y_test)
preds = bnb.predict(X_train)
print(np.unique(preds, return_counts=True))
print(bnbScore)

#### Logistic Regression (binary)

In [ ]:
# Grid Search on Logistic Regression
parameters = {'solver':('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')}
lr = GridSearchCV(LogisticRegression(), parameters)
lr.fit(X_train, y_train)
estimator = lr.best_estimator_
print(estimator)
clf = estimator
clf.predict(X_test)
clfscore = clf.score(X_test, y_test)
print(clfscore)

#### Recurrent Neural Network

In [ ]:
# Note: this portion is taken from our HW2 submission
def make_model(activation_function, num_hidden_layers, hidden_layer_size): 
  model = Sequential()

    # Single layer model
  if num_hidden_layers == 0: # then just specify a single layer, 1 is size of output
        model.add(Dense(1, 
                        input_dim=X_train.shape[1], 
                        activation=activation_function,
                        use_bias=True))
        model.add(Dropout(0.5))
    
    # Specify initial layer with a hidden layer
  if num_hidden_layers >= 1: 
        model.add(Dense(hidden_layer_size, 
                        input_dim=X_train.shape[1], 
                        activation=activation_function,
                        use_bias=True))
        model.add(Dropout(0.5))
    
    # Now add additional hidden layers
  for i in range(0,num_hidden_layers-1):
        model.add(Dense(hidden_layer_size, 
                        activation=activation_function, 
                        use_bias=True))
        model.add(Dropout(0.5))
    
  if num_hidden_layers > 0:       
        model.add(Dense(1)) # Final output layer, don't add if no hidden layers

  model.compile(loss='mean_squared_error',
                  optimizer='adam')
  return model

In [ ]:
classifier = KerasRegressor(make_model, batch_size=32, epochs=1)

params = [{'num_hidden_layers': [0],
          'hidden_layer_size': [0],
          'activation_function': ['linear', 'sigmoid', 'relu', 'tanh']},
          {'num_hidden_layers': [1,2,3],
          'hidden_layer_size': [64, 128, 256],
          'activation_function': ['linear', 'sigmoid', 'relu', 'tanh']}]


In [ ]:
xNew = newDf.drop(['degree'], axis=1)
xNew = xNew.drop(['badApple'], axis=1)
xNew = xNew.drop(['County'], axis=1)
xNew = xNew.drop(['Name'], axis=1)

# Perform Grid Search on neural network
# WARNING: MIght overwhelm and break Kernel!
X_train, X_test, y_train, y_test = train_test_split(xNew, newDf['badApple'], test_size=0.20, random_state=12345)
grid = GridSearchCV(classifier,
                         param_grid=params,
                         scoring='neg_mean_squared_error', #sklearn optimizing by maximizing negative MSE
                         n_jobs=1,
                         verbose=2,
                         cv=5,# Number of folds for CV
                     
                   )

# Fit Model
X_train = np.asarray(X_train).astype('float32')
grid.fit(X_train, y_train)

# Evaluate
print('The parameters of the best model are: ')
print(grid.best_params_)


best_model = grid.best_estimator_ #scikit-wrapped best model
preds = best_model.predict(X_test)
clfscore = bets_model.score(X_test, y_test)
print(clfscore)

In [ ]:
# Neural Network predictiveness(93.9%)
preds = clf.predict(X_test)
clfscore = clf.score(X_test, y_test)
print(clfscore)
print(np.unique(preds, return_counts=True))

In [ ]:
# Evaluate Paramters of best model 
print('The parameters of the best model are: ')
print(grid.best_params_)

best_model = grid.best_estimator_ #scikit-wrapped best model

# Additional evaluation metrics
X_test = np.asarray(X_test).astype('float32')
preds = best_model.predict(X_test)
preds=np.rint(preds)
print(preds)
rmse=np.sqrt(np.mean((y_test-preds)**2))
clfscore = best_model.score(X_test, y_test)
print(rmse)


#### Lasso Feature Selection

In [ ]:
# Train model using CV
X_train, X_test, y_train, y_test = train_test_split(xNew[['Force', 'Offensive Language', 'Discourtesy', 'Abuse of Authority']], newDf['badApple'], test_size=0.20, random_state=12345)
lasso = LassoCV().fit(X_train, y_train)

# Print feature importances
importance = np.abs(lasso.coef_)
feature_names = np.array(X_train.columns)
plt.bar(height=importance, x=feature_names)
plt.title("Feature importances via coefficients")
plt.show()

# Detailed feature importances
for i in range(0, len(importance)):
    print(str(importance[i]) + ' ' + feature_names[i])
    

# Lasso Evaluation
print(lasso.score(X_test, y_test))

X_test = np.asarray(X_test).astype('float32')
preds = lasso.predict(X_test)
preds=np.rint(preds)
print(preds)

# Evaluate false pos/neg
i = 0
falsePos = 0
falseNeg = 0
correct = 0
for y in y_test:

    if preds[i] == y:
        if y == 1:
        correct += 1
    elif preds[i] > y:
        falsePos += 1
    else:
        falseNeg += 1
    i += 1
    

# Print Findings
print(correct/i)
print(falsePos/i)
print(falseNeg/i)

rmse=np.sqrt(np.mean((y_test-preds)**2))
clfscore = lasso.score(X_test, y_test)
print(rmse)


### Subgraph Analysis

In [ ]:
# See percentage of complaints caused by certain subgropus of NetworkX graph 
file1 = r"subgraph.csv"
subgraph = pd.read_csv(file1)


# ACTION: Choose to be analyzed sugraph; comment out one of the ids!

# 22k officers in largest subgraph of NetworkX graph
ids = subgraph['Unique ID']

# Worst 22 instigators from NetworkX graph
ids = [56121, 39959, 16205,48065, 49153, 48223,80598, 38157, 55704, 14854, 54730, 48343, 38305, 38148, 54534, 43801,15858, 55188, 19736, 72537,
       55991, 48086]


# Initially, sum all complaint IDs where officers of ids array are involved 
counter = 0
no = 0
complaints = []
for row in df.iterrows():
 
    if row[1][0] in ids:
        # if len(row) < 5: continue
        if math.isnan(row[1][5]):
            counter += 1
        else:
            complaints.append(int(row[1][5]))

# Next, sum up total number of complaints caused by given complaint ids
for comp in df['Complaint Id']:
    if math.isnan(comp): continue
    if int(comp) in complaints:
        counter += 1
    else:
        no += 1

# Print out percentage of total complaints subgraph is responsible for
print(counter / (counter + no))
print(no)    

# Interesting findings can be found in paper

# Predictions

In [ ]:
# Seperate X and y
copyDf = xNew.copy()
xNew = xNew[xNew['wg15'] > 0] 
xPred = xNew['wg16'] # to be predicted
xNew = xNew.drop(['wg16'], axis=1)
xNew  xNew.drop(['wg17'], axis=1)
xNew = xNew.drop(['wg18'], axis=1)

In [ ]:
# Use Linear regression to predict wages in 2016 using past wages, complaints, rank, ...

# Randomized train test split
X_train, X_test, y_train, y_test = train_test_split(
    xNew, xPred, test_size=0.33, random_state=42)

# Grid Search on LinReg
linreg = LinearRegression(normalize = False)
lin_params = [{'fit_intercept': [True, False], 'normalize': [True, False]}]
grid = GridSearchCV(linreg,
                         param_grid=lin_params,
                         scoring='neg_mean_squared_error', #sklearn optimizing by maximizing negative MSE
                         n_jobs=1,
                         verbose=2,
                         cv=5)

# Choose which model to fit on:

# 1. Grid Search
grid.fit(X_train, y_train)
prediction_df = grid.predict(X_test)

# 2. Reg LinearRegression
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
prediction_df = regr.predict(X_test)

# Evaluation: 

# The coefficients (printed later)
# print('Coefficients: \n', grid.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, prediction_df))
# The coefficient of determination: 
print('Coefficient of determination: %.2f'
      % r2_score(y_test, prediction_df))

# Print Coefficients with column names
columns = xNew.columns
for i in range(0, len(columns)):
    print(columns[i] + " " + str(regr.coef_[i]))

prediction_df

# PCA & MCA

## MCA on original data set

NOTE: This section analyzes the original complaint data set. No interesting findings found. For the optimized PCA, see the section on "PCA on Improved Data Set", which uses our own data set and is much more promising.

In [ ]:
#Fit MCA on complaint data set 
mca_df = mca.MCA(df[, cols=None][, ncols=None][, benzecri=True][, TOL=1e-4])
principalComponents = pca.fit_transform(df)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])


In [ ]:
# Print resulting matrix & table
mca_ben = mca.MCA(df)
mca_ind = mca.MCA(df, benzecri=False)
data = {'Iλ': pd.Series(mca_ind.L),
        'τI': mca_ind.expl_var(greenacre=False, N=4),
        'Zλ': pd.Series(mca_ben.L),
        'τZ': mca_ben.expl_var(greenacre=False, N=4),
        'cλ': pd.Series(mca_ben.L),
        'τc': mca_ind.expl_var(greenacre=True, N=4)}

# 'Indicator Matrix', 'Benzecri Correction', 'Greenacre Correction'
columns = ['Iλ', 'τI', 'Zλ', 'τZ', 'cλ', 'τc']
table2 = pd.DataFrame(data=data, columns=columns).fillna(0)
table2.index += 1
table2.loc['Σ'] = table2.sum()
table2.index.name = 'Factor'

# Print table
table2

In [ ]:
# Fit MCA solely on Rank, Command, & Allegation
dfCopy = df[['Rank', 'Command', 'Allegation']]
mca.fit(dfCopy)

In [ ]:
# Plot coordinates - 1
mca.plot_coordinates(
     X=dfCopy,
     ax=None,
     figsize=(6, 6),
     show_row_points=True,
     row_points_size=10,
     show_row_labels=False,
     show_column_points=True,
     column_points_size=30,
     show_column_labels=False,
     legend_n_cols=1
 )

In [ ]:
# Plot coordinates - 2
mca.plot_coordinates(dfCopy,
                     row_points_alpha=.2,
                     figsize=(10, 10),
                     show_column_labels=True
                    );

In [ ]:
# Show MCA Grayscale Figure
points = table3.loc[fs].values
labels = table3.columns.values

plt.figure()
plt.margins(0.1)
plt.axhline(0, color='gray')
plt.axvline(0, color='gray')
plt.xlabel('Factor 1')
plt.ylabel('Factor 2')
plt.scatter(*points, s=120, marker='o', c='r', alpha=.5, linewidths=0)
for label, x, y in zip(labels, *points):
    plt.annotate(label, xy=(x, y), xytext=(x + .03, y + .03))
plt.show()


## PCA on Improved Data Set

### Initial PCA

In [ ]:
# Read in our cleaned data set (attached to paper in Zip file)
file1 = r"/content/drive/Shareddrives/COS424/hw3/cleanData.csv" # our own improved data set (can be exported above)
df = pd.read_csv(file1)

In [ ]:
# Remove non-numeric olumns
del df['Name']
del df['County']

# Add two new columns
df['Complaint Code'] = 0 # Number of complaints in categorical form (0 - lowest // 3 - highest)
df['Guilty'] = 0 # Number of times found guilty (0 - lowest // 3 - highest)

# Fill categorical columns
counter = 0
for row in df.iterrows():
    index = row[1][0]
    if row[1][32]==0: df['Complaint Code'][index] = 0
    elif row[1][32]<5: df['Complaint Code'][index] = 1
    elif row[1][32]<20: df['Complaint Code'][index] = 2
    else: df['Complaint Code'][index] = 3

for row in df.iterrows():
    index = row[1][0]
    if row[1][69]==0: df['Guilty'][index] = 0
    elif row[1][69]<5: df['Guilty'][index] = 1
    elif row[1][69]<20: df['Guilty'][index] = 2
    else: df['Guilty'][index] = 3

In [ ]:
x = dfCopy.values
# Standardizing the features
x = StandardScaler().fit_transform(x)

# PCA with 2 components
pca = PCA(n_components=2)

principalComponents = pca.fit_transform(x)

In [ ]:
# Print explained variance and ratio
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
print(pca.explained_variance_)
print(pca.explained_variance_ratio_)

In [ ]:
# Print figure using components
fig = plt.figure()

ax = plt.axes()
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
ax.scatter(principalDf['principal component 1'],principalDf['principal component 2'])

# Print plt figure
plt.scatter(principalDf['principal component 1'],principalDf['principal component 2'], alpha=.1, color='black')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')

In [ ]:
# Add rank to principalDf with 2 components
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, df['Rank']], axis = 1)
finalDf.head()

In [ ]:
# KMeans Inertia Analysis
ks = range(1, 10)
inertias = []
for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k)
    
    # Fit model to samples
    model.fit(principalComponents)
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)
    
# Plot inertia findings (in paper)
plt.plot(ks, inertias, '-o', color='black')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

In [ ]:
# Evaluate KMeans Performance and print findings
kmeans_pca=KMeans(n_clusters=4, init='k-means++', random_state=42)
kmeans_pca.fit(principalComponents)
cluster = kmeans_pca.labels_
print(cluster)
clusterDF = pd.DataFrame(cluster)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, clusterDF], axis = 1)
print(finalDf[finalDf.columns[2]])

In [ ]:
# This section is inspired from:
#https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60

# Print PCA in visualy pleasing way
# Note: this section can be adjusted to fit different targets.

fig = plt.figure(figsize = (8,8)) 
ax = plt.axes()
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('3 component PCA', fontsize = 20)
targets = [1, 2,3,4]
colors = ['c','m','r','b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf[finalDf.columns[2]] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 1)
ax.legend(targets)
ax.grid()

In [ ]:
pca1 = pca.components_[0]
pca2 = pca.components_[1]

# Help on this section came from:
#https://stackoverflow.com/questions/6910641/how-do-i-get-indices-of-n-maximum-values-in-a-numpy-array
ind1 = np.argpartition(pca1, -5)[-5:]
ind2 = np.argpartition(pca2, -5)[-5:]

### Additional 2D PCA

Note: The remainder this section is a repeat of the previous section with an additionl column and different targets. There is a strong overlap in code and thus commenting is kept to a minimum.

In [ ]:
# Create new complaints column (make continuous into categorical)
# 0 - lowest // 3 - highest number of compliants
df['Complaints']=0
comp = 0
for row in df.iterrows():
    index = row[1][0]
    if row[1][32]==0: df['Complaints'][index] = 0
    elif row[1][32]<5: df['Complaints'][index] = 1
    elif row[1][32]<20: df['Complaints'][index] = 2
    else: df['Complaints'][index] = 3

In [ ]:
# Perform PCA again with new column added
x = dfRank.values
# Standardizing the features
x = StandardScaler().fit_transform(x)

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])


In [ ]:
# Same as above section with new 
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, df['Complaints']], axis = 1)
finalDf.head()

In [ ]:
fig = plt.figure(figsize = (8,8)) 
ax = plt.axes()
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = [0,1,2,
           3]
colors = ['c','m','r','g']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['Complaints'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
        
               , c = color
               , s = 1)
ax.legend(targets)
ax.grid()

In [ ]:
from sklearn.cluster import KMeans
ks = range(1, 10)
inertias = []
for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k)
    
    # Fit model to samples
    model.fit(principalComponents)
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)
    
plt.plot(ks, inertias, '-o', color='black')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

In [ ]:
kmeans_pca=KMeans(n_clusters=2, init='k-means++', random_state=42)
kmeans_pca.fit(principalComponents)
cluster = kmeans_pca.labels_
print(cluster)

# Additional Non-Essential Code

## Basic Data Analysis

The following section is left uncommented as none of its findings are relevant to our paper. As it was included in the starter notebook and helped us be familiarized with the data, however, we feel obliged to keep it in our notebook for transparency purposes. It was not coded by us.

In [ ]:
officer_counts = df['Full Name'].value_counts().reset_index()
officer_counts.columns = ['Officer', 'Number of complaints']
officer_counts.head(10)

In [ ]:
fig, ax = plt.subplots(1, 3)
fig.set_size_inches(18, 5)

hist = officer_counts.hist(bins=100, range = [0, 20], ax=ax[0])
hist = officer_counts.hist(bins=100, range = [20, 50], ax=ax[1])
hist = officer_counts.hist(bins=100, range = [50, 200], ax=ax[2])

In [ ]:
allegations = df['Allegation'].value_counts().reset_index()
allegations.columns = ['Allegation', 'Count of allegation']
allegations.head(10)

In [ ]:
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(15, 5)

years_of_complaints = df.groupby(["year"]).size().reset_index(name="Number of Complaints per Year").sort_values(by=['year'], ascending = True)

years_of_complaints.plot(kind='bar',x='year',y='Number of Complaints per Year', ax=ax[0])

months_of_complaints = df.groupby(["month"]).size().reset_index(name="Number of Complaints per Month").sort_values(by=['month'], ascending = True)
months_of_complaints.plot(kind='bar',x='month',y='Number of Complaints per Month', ax=ax[1])

years = df['year'].unique()
years = [x for x in years if (math.isnan(x) == False)]
years.sort()

## Example use of Matrix Factorization

The following section is left uncommented as none of its findings are relevant to our paper. As it was included in the starter notebook and helped us be familiarized with the data, however, we feel obliged to keep it in our notebook for transparency purposes. It was not coded by us.

In [ ]:
officers    = np.array(df['Full Name']).astype(str)
allegations = np.array(df['Allegation']).astype(str)

# Convert from officer names and allegation names to label encoding.
le1 = preprocessing.LabelEncoder()
le1.fit(officers)

le2 = preprocessing.LabelEncoder()
le2.fit(allegations)

xs = le1.transform(officers)
ys = le2.transform(allegations)

assert len(xs) == len(ys)

In [ ]:
# Get counts of each officer/allegation interaction.
non_unique_edges = zip(xs, ys)
unique_edges_with_counts = pd.Series(non_unique_edges).value_counts()

edges  = unique_edges_with_counts.index.values
counts = unique_edges_with_counts.values

u_xs, u_ys = zip(*edges)

# Construct sparse matrix data type.
X = csr_matrix((counts, (u_xs, u_ys)))

# Compute Non-negative Matrix Factorization.
n_components = 5
nmf = NMF(n_components=n_components, random_state=41)
nmf.fit(X)

In [ ]:
print(f'Top {n_components} components lead to reconstruction error (in Frobenius norm) of {nmf.reconstruction_err_}.')
print()

topK = 5
for i in range(nmf.components_.shape[0]):
  print(f'Top allegations in component {i+1}:')
  allegs = np.argsort(nmf.components_[i])[::-1][:topK]
  for j in range(topK):
    print(f'\t{le2.classes_[allegs[j]]}')
  print()

In [ ]:
# Compute truncated SVD (uses a randomized algorithm by default).
n_components = 5
svd = TruncatedSVD(n_components=n_components, random_state=41)
svd.fit(X)

# Look at the top principle components.
print(f'Top {n_components} components explain {svd.explained_variance_ratio_.sum()} of the variance.')
print(f'Breakdown: {svd.explained_variance_ratio_}\n')

## CapStat.NYC Police Officer Database
Compiled from CapStat.NYC by Wendy Ho

https://github.com/wendyho/NYPD-Misconduct-Complaint-Database

In [ ]:
df3_nameseries = df3['Full Name'].values.astype(str)
df_nameseries  = df['Full Name'].values.astype(str)

df_match = df.loc[df['Full Name'].isin(df3_nameseries)]
df3_match = df3.loc[df3['Full Name'].isin(df_nameseries)]
print(f'{len(df_match)} of the {len(df)} complaints matched to a police officer in the CapStat.NYC database.')
print(f'{len(df3_match)} of the {len(df3)} police officers have at least one complaint against them.')

In [ ]:
df3['Rank'].value_counts()

In [ ]:
df3['Location'].value_counts()

In [ ]:
#---------------------------- Run Time Params --------------------------------#

# Probably going to try and use - flag conventions with __init__(self, *args, **kwargs)


#---------------------------- Load Data --------------------------------------#
num_people = 25
num_groups = 5
alpha = np.ones(num_groups).ravel()*0.1
#B = np.eye(num_groups)*0.85
#B = B + np.random.random(size=[num_groups,num_groups])*0.1

B = np.eye(num_groups)*0.8
B = B + np.ones([num_groups,num_groups])*0.2-np.eye(num_groups)*0.2

#---------------------------- Setup Model -----------------------------------#
raw_model = create_model(nmf.components_, num_people, num_groups, alpha, B)
#model_instance = pymc.Model(raw_model)

#---------------------------- Call MAP to initialize MCMC -------------------#
#pymc.MAP(model_instance).fit(method='fmin_powell')
print('---------- Finished Running MAP to Set MCMC Initial Values ----------')
#---------------------------- Run MCMC --------------------------------------#
print('--------------------------- Starting MCMC ---------------------------')
M = pymc.MCMC(raw_model)
M.sample(100000,50000, thin=5, verbose=0)

In [ ]:
file4 = r"/content/drive/Shareddrives/COS424/hw3/capstat_expanded_dataset.csv" 
df4 = pd.read_csv(file4)
df4.head(10)

In [ ]:
# Declaring Model
model = KMeans(n_clusters=3)

# Fitting Model
model.fit(nmf.components_)

# Predicitng a single input
all_predictions = model.predict(nmf.components_)

# Printing Predictions
print(all_predictions)

In [ ]:

#filter rows of original data
filtered_label0 = df[label == 0]
 
#plotting the results
plt.scatter(filtered_label0[:,0] , filtered_label0[:,1])
plt.show()

In [ ]:
#Getting the Centroids
centroids = model.cluster_centers_
u_labels = np.unique(all_predictions)
 
#plotting the results:
 
for i in u_labels:
    plt.scatter(df[all_predictions == i , 0] , df[all_predictions == i , 1] , 
                all_predictions = i)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'k')
plt.legend()
plt.show()